## Computing the Worm Spline (Julia)

The spline computation runs in **Julia** using the Flavell lab's `BehaviorDataNIR.jl` package. It:
1. Segments the worm from each NIR frame using a UNet2D model
2. Computes the medial axis (skeleton) of the worm
3. Fits a smooth spline along the medial axis
4. Detects omega turns and self-intersections
5. Recomputes a cleaned spline

julia 1.8.1 (might be under jupyter kernels)

In [ ]:
# using PyCall
# println(PyCall.python)

C:\Users\munib\miniconda3\envs\nir-pipeline\python.exe


In [ ]:
# import Pkg; Pkg.add("Conda")

    Updating registry at `C:\Users\munib\.julia-flv\registries\General.toml`
   Resolving package versions...
    Updating `C:\Users\munib\.julia-flv\environments\v1.8\Project.toml`
  [8f4d0f93] + Conda v1.10.3
  No Changes to `C:\Users\munib\.julia-flv\environments\v1.8\Manifest.toml`


In [23]:
# using Conda
# Conda.add("scikit-image", channel="conda-forge")
# Conda.add("scikit-learn", channel="conda-forge")
# Conda.add("networkx")

[ Info: Running `conda install -y -c conda-forge scikit-image` in root environment


Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: done




==> WARNING: A newer version of conda exists. <==
    current version: 25.11.0
    latest version: 26.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda





## Package Plan ##

  environment location: C:\Users\munib\.julia-flv\conda\3\x86_64

  added / updated specs:
    - scikit-image


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    importlib-metadata-8.7.0   |     pyhe01879c_1          34 KB  conda-forge
    lazy-loader-0.4            |     pyhd8ed1ab_2          16 KB  conda-forge
    networkx-3.6.1             |     pyhcf101f3_0         1.5 MB  conda-forge
    scikit-image-0.26.0        |np2py312h9ea65bc_0        19.8 MB  conda-forge
    scipy-1.17.0               |  py312h9b3c559_1        14.2 MB  conda-forge
    zipp-3.23.0                |     pyhcf101f3_1          24 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        35.6 MB

The following NEW packages will be INSTALLED:

  importlib-metadata conda-forge/noarch::importlib-metadata-8.7.0-pyhe01879c

[ Info: Running `conda install -y -c conda-forge scikit-learn` in root environment


...


## Package Plan ##

  environment location: C:\Users\munib\.julia-flv\conda\3\x86_64

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    joblib-1.5.3               |     pyhd8ed1ab_0         221 KB  conda-forge
    scikit-learn-1.8.0         |np2py312hea30aaf_1         8.5 MB  conda-forge
    threadpoolctl-3.6.0        |     pyhecae5ae_0          23 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         8.7 MB

The following NEW packages will be INSTALLED:

  joblib             conda-forge/noarch::joblib-1.5.3-pyhd8ed1ab_0 
  scikit-learn       conda-forge/win-64::scikit-learn-1.8.0-np2py312hea30aaf_1 
  threadpoolctl      conda-forge/noarch::threadpoolctl-3.6.0-pyhecae5ae_0 



scikit-learn-1.8.0   | 8.5 MB    |            |   0% 
joblib-1.5.3         | 221 KB 

[ Info: Running `conda install -y networkx` in root environment


Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: done




==> WARNING: A newer version of conda exists. <==
    current version: 25.11.0
    latest version: 26.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda





# All requested packages already installed.



In [ ]:
# using PyCall
# # Try to import them
# ski = pyimport("skimage.morphology")
# skl = pyimport("sklearn.neighbors")
# nx = pyimport("networkx")
# println("All packages imported successfully!")

All packages imported successfully!


# SETUP

In [2]:
# Expand the path to include all Conda binary locations
conda_env = "C:\\Users\\munib\\miniconda3\\envs\\nir-pipeline"
new_paths = [
    joinpath(conda_env, "bin"),
    joinpath(conda_env, "Library", "bin"),
    joinpath(conda_env, "Scripts")
]
ENV["PATH"] = join(new_paths, ";") * ";" * ENV["PATH"]

using PyCall
# Re-verify PyCall is definitely looking at the right python
println("PyCall using: ", PyCall.python) 

torch = pyimport("torch")

PyCall using: C:\Users\munib\miniconda3\envs\nir-pipeline\python.exe


PyObject <module 'torch' from 'C:\\Users\\munib\\miniconda3\\envs\\nir-pipeline\\lib\\site-packages\\torch\\__init__.py'>

In [3]:
using Pkg
ENV["PYTHON"] = "C:\\Users\\munib\\miniconda3\\envs\\nir-pipeline\\python.exe"
Pkg.build("PyCall")

    Building Conda ─→ `C:\Users\munib\.julia-flv\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\8f06b0cfa4c514c7b9546756dbae91fcfbc92dc9\build.log`
    Building PyCall → `C:\Users\munib\.julia-flv\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\9816a3826b0ebf49ab4926e2b18842ad8b5c8f04\build.log`


In [4]:
using PyCall
PyCall.python

"C:\\Users\\munib\\miniconda3\\envs\\nir-pipeline\\python.exe"

In [5]:
torch = pyimport("torch")

println("Is CUDA available? ", torch.cuda.is_available())
println("GPU Name: ", torch.cuda.get_device_name(0))

Is CUDA available? true
GPU Name: NVIDIA GeForce RTX 5080


# CONFIG

In [ ]:
# --- GPU setup (set to "" to disable GPU) ---
ENV["CUDA_VISIBLE_DEVICES"] = "0"   # Use GPU 0; set to "" for CPU
ENV["JULIA_IO_BUFFER"] = "0"

# --- Load packages ---
using ImageDataIO, BehaviorDataNIR, UNet2D, H5Zblosc
using HDF5, PyPlot, FileIO

In [8]:
# ============================================================
# USER CONFIG — EDIT THESE
# ============================================================
NAME = "date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm002"  

# Windows paths:
PATH_H5   = "C:\\Users\\munib\\POSTDOC\\DATA\\g5ht-free\\20251028\\$(NAME).h5"
PATH_JLD2 = "C:\\Users\\munib\\POSTDOC\\DATA\\g5ht-free\\20251028\\$(NAME)_data_dict.jld2"

# Linux paths (if running on server):
# PATH_H5   = "/data3/albert/2024/$(NAME).h5"
# PATH_JLD2 = "/home/user/data/$(NAME)_data_dict.jld2"

# Path to segmentation model weights
# (copy worm_segmentation_best_weights_0310.pt to a known location)
path_weight = "C:\\Users\\munib\\POSTDOC\\CODE\\g5ht-pipeline\\nir\\worm_segmentation_best_weights_0310.pt"

"C:\\Users\\munib\\POSTDOC\\CODE\\g5ht-pipeline\\nir\\worm_segmentation_best_weights_0310.pt"

In [9]:
# ============================================================
# GET FRAME COUNT
# ============================================================
# Simple approach:
MAX_T_NIR = size(h5open(PATH_H5)["img_nir"])[3]

# Safer approach — finds last readable frame (use if h5 file is truncated):
# function last_good_frame(PATH_H5)
#     out = 0
#     for i = 1:size(h5open(PATH_H5)["img_nir"])[3]
#         try
#             img = h5open(PATH_H5)["img_nir"][:, :, i]
#         catch
#             break
#         end
#         out = i
#     end
#     return out
# end
# MAX_T_NIR = last_good_frame(PATH_H5)

println("Total NIR frames: $MAX_T_NIR")

Total NIR frames: 7974


In [10]:
# ============================================================
# CREATE `param` DICTIONARY
# ============================================================
# These are the algorithm parameters — usually no need to change
param = Dict()

# --- For 1st compute_worm_spline!() ---
param["num_center_pts"]           = 1000    # Number of points along the spline
param["img_label_size"]           = (480, 360)  # Expected NIR image size
param["nose_confidence_threshold"] = 0.99   # UNet confidence for nose detection
param["nose_crop_threshold"]      = 20      # Crop threshold for nose region

# --- For compute_worm_thickness() ---
param["min_len_percent"]          = 90      # Min worm length percentile
param["max_len_percent"]          = 98      # Max worm length percentile

# --- For 2nd compute_worm_spline!() (after thickness detection) ---
param["worm_thickness_pad"]       = 3       # Padding around worm thickness
param["boundary_thickness"]       = 5       # Boundary thickness for detection
param["close_pts_threshold"]      = 30      # Threshold for close points
param["trim_head_tail"]           = 15      # Trim head/tail pixels
param["max_med_axis_delta"]       = Inf     # Max medial axis displacement

Inf

In [11]:
# ============================================================
# CREATE `data_dict` DICTIONARY
# ============================================================
# This is the main output dictionary. Here are ALL the keys
# and what they store:

data_dict = Dict()

# --- Medial axis results (Dict of per-frame data) ---
data_dict["med_axis_dict"] = Dict()       # Medial axis points per frame
data_dict["med_axis_dict"][0] = nothing    # Initialize with sentinel

# --- Point ordering (Dict of per-frame data) ---
data_dict["pts_order_dict"] = Dict()      # Ordered skeleton points per frame
data_dict["pts_order_dict"][0] = nothing   # Initialize with sentinel

# --- Omega turn detection (Dict of Bool per frame) ---
data_dict["is_omega"] = Dict()            # Whether frame has omega turn

# --- Spline coordinates: (n_frames, num_center_pts+1) matrices ---
data_dict["x_array"] = zeros(MAX_T_NIR, param["num_center_pts"] + 1)
data_dict["y_array"] = zeros(MAX_T_NIR, param["num_center_pts"] + 1)

# --- Per-frame scalar metrics ---
data_dict["nir_worm_angle"] = zeros(MAX_T_NIR)  # Worm body angle per frame
data_dict["eccentricity"]   = zeros(MAX_T_NIR)  # Worm eccentricity per frame

# --- Error tracking ---
error_dict = Dict()

Dict{Any, Any}()

# MAIN

In [ ]:
# ============================================================
# RUN SPLINE COMPUTATION (3 stages)
# ============================================================

# Stage 1: Load the UNet2D segmentation model
println("Loading model...")
using PyCall
# Define the CPU device using PyCall to talk to torch
# device = pyimport("torch").device("cpu")
# worm_seg_model = create_model(1, 1, 16, path_weight, device=device)
worm_seg_model = create_model(1, 1, 16, path_weight)

#  expliclty move worm_seg_model to CPU (should already be on CPU, but just to be sure)
# worm_seg_model.to(device)

Loading model...


PyObject UNet2D(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(32, eps=1e-05, 

In [19]:
# find the path to the compute_worm_spline!() function
@which compute_worm_spline!(
    param, PATH_H5, worm_seg_model, nothing,
    data_dict["med_axis_dict"], data_dict["pts_order_dict"],
    data_dict["is_omega"], data_dict["x_array"], data_dict["y_array"],
    data_dict["nir_worm_angle"], data_dict["eccentricity"],
    timepts=1:MAX_T_NIR
)

(::BehaviorDataNIR.var"#compute_worm_spline!##kw")(::Any, ::typeof(compute_worm_spline!), param, path_h5, worm_seg_model, worm_thickness, med_axis_dict, pts_order_dict, is_omega_dict, x_array, y_array, nir_worm_angle, eccentricity) in BehaviorDataNIR at C:\Users\munib\.julia-flv\packages\BehaviorDataNIR\SRM4J\src\segmentation.jl:292

In [25]:
# Stage 2: Initial spline computation, around X minutes
println("Computing initial spline (pass 1)...")
error_dict["worm_spline_errors_1"] = compute_worm_spline!(
    param, PATH_H5, worm_seg_model, nothing,
    data_dict["med_axis_dict"], data_dict["pts_order_dict"],
    data_dict["is_omega"], data_dict["x_array"], data_dict["y_array"],
    data_dict["nir_worm_angle"], data_dict["eccentricity"]
)

Computing initial spline (pass 1)...


Dict{Any, Any} with 7974 entries:
  4986 => ArgumentError("ref of NULL PyObject")
  7329 => ArgumentError("ref of NULL PyObject")
  4700 => ArgumentError("ref of NULL PyObject")
  4576 => ArgumentError("ref of NULL PyObject")
  7144 => ArgumentError("ref of NULL PyObject")
  6073 => ArgumentError("ref of NULL PyObject")
  2288 => ArgumentError("ref of NULL PyObject")
  1703 => ArgumentError("ref of NULL PyObject")
  1956 => ArgumentError("ref of NULL PyObject")
  2350 => ArgumentError("ref of NULL PyObject")
  7685 => ArgumentError("ref of NULL PyObject")
  5975 => ArgumentError("ref of NULL PyObject")
  3406 => ArgumentError("ref of NULL PyObject")
  2841 => ArgumentError("ref of NULL PyObject")
  2876 => ArgumentError("ref of NULL PyObject")
  687  => ArgumentError("ref of NULL PyObject")
  7353 => ArgumentError("ref of NULL PyObject")
  185  => ArgumentError("ref of NULL PyObject")
  1090 => ArgumentError("ref of NULL PyObject")
  2015 => ArgumentError("ref of NULL PyObject")
  7272

In [ ]:
error_dict["worm_spline_errors_1"][4986]

Dict{Any, Any}(4986 => ArgumentError("ref of NULL PyObject"), 7329 => ArgumentError("ref of NULL PyObject"), 4700 => ArgumentError("ref of NULL PyObject"), 4576 => ArgumentError("ref of NULL PyObject"), 7144 => ArgumentError("ref of NULL PyObject"), 6073 => ArgumentError("ref of NULL PyObject"), 2288 => ArgumentError("ref of NULL PyObject"), 1703 => ArgumentError("ref of NULL PyObject"), 1956 => ArgumentError("ref of NULL PyObject"), 2350 => ArgumentError("ref of NULL PyObject"), 7685 => ArgumentError("ref of NULL PyObject"), 5975 => ArgumentError("ref of NULL PyObject"), 3406 => ArgumentError("ref of NULL PyObject"), 2841 => ArgumentError("ref of NULL PyObject"), 2876 => ArgumentError("ref of NULL PyObject"), 687 => ArgumentError("ref of NULL PyObject"), 7353 => ArgumentError("ref of NULL PyObject"), 185 => ArgumentError("ref of NULL PyObject"), 1090 => ArgumentError("ref of NULL PyObject"), 2015 => ArgumentError("ref of NULL PyObject"), 7272 => ArgumentError("ref of NULL PyObject"), 

In [15]:
# Stage 3: Detect self-intersections using worm thickness
println("Detecting self-intersection...")
data_dict["worm_thickness"], count = compute_worm_thickness(
    param, PATH_H5, worm_seg_model,
    data_dict["med_axis_dict"], data_dict["is_omega"]
)

Detecting self-intersection...


LoadError: ArgumentError: empty data vector

In [ ]:
# Stage 4: Recompute spline with thickness correction
println("Recomputing spline (pass 2)...")
error_dict["worm_spline_errors_2"] = compute_worm_spline!(
    param, PATH_H5, worm_seg_model, data_dict["worm_thickness"],
    data_dict["med_axis_dict"], data_dict["pts_order_dict"],
    data_dict["is_omega"], data_dict["x_array"], data_dict["y_array"],
    data_dict["nir_worm_angle"], data_dict["eccentricity"],
    timepts=1:MAX_T_NIR
)

# ============================================================
# SAVE RESULTS
# ============================================================
println("Saving data_dict to: $PATH_JLD2")
save(PATH_JLD2, "data_dict", data_dict)
println("Done!")

In [ ]:
# Quick sanity check — plot every 100th frame's spline
using PyPlot
figure(figsize=(10, 8))
step = 100
for i = 1:step:MAX_T_NIR
    plot(data_dict["x_array"][i, :], data_dict["y_array"][i, :])
end
title("Worm splines (every $(step)th frame)")
xlabel("x (pixels)")
ylabel("y (pixels)")
savefig("spline_check.png", dpi=150)

### `data_dict` Keys After Spline Computation

After running the spline script, the saved `.jld2` file contains:

| Key | Type | Shape | Description |
|-----|------|-------|-------------|
| `x_array` | `Matrix{Float64}` | `(MAX_T_NIR, 1001)` | X coords of spline points per frame |
| `y_array` | `Matrix{Float64}` | `(MAX_T_NIR, 1001)` | Y coords of spline points per frame |
| `nir_worm_angle` | `Vector{Float64}` | `(MAX_T_NIR,)` | Overall worm body angle per frame |
| `eccentricity` | `Vector{Float64}` | `(MAX_T_NIR,)` | Worm shape eccentricity per frame |
| `med_axis_dict` | `Dict` | per-frame | Medial axis points (raw skeleton) |
| `pts_order_dict` | `Dict` | per-frame | Ordered skeleton points |
| `is_omega` | `Dict{Int,Bool}` | per-frame | Whether frame shows omega turn |
| `worm_thickness` | value | scalar | Estimated worm thickness in pixels |